<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/pyspark_clusteringOnNumericalAndVector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pyspark
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=bc0f7e5f9e7e9a41b76e816cbdac70507f17e5f9448833b5461008f5b616f213
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [19]:
spark = SparkSession.builder.appName("ClusteringExample").getOrCreate()
spark

In [ ]:
# data = [
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ]
# df = spark.createDataFrame(data, ["feature1", "vector", "category"])
# category_indexer = StringIndexer(inputCol="category", outputCol="category_index")

#org: name	main_category	sub_category	image	link	ratings	no_of_ratings	discount_price	actual_price
#sel: name main_category sub_category ratings no_of_ratings discount_price actual_price



In [ ]:
assembler = VectorAssembler(
    inputCols=["feature1", "vector"],
    outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[category_indexer, assembler, kmeans])

model = pipeline.fit(df)

predictions = model.transform(df)

In [ ]:
predictions.show()

+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



In [ ]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))
predictions.show()

Silhouette with squared euclidean distance = 0.6120640643838152
+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



### 寵物領養率的數據分析

* PetID - Unique hash ID of pet profile
* AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
* Type - Type of animal (1 = Dog, 2 = Cat)
* Name - Name of pet (Empty if not named)
* Age - Age of pet when listed, in months
* Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
* Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
* Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
* Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
* Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
* Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
* MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
* FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
* Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
* Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
* Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
* Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
* Quantity - Number of pets represented in profile
* Fee - Adoption fee (0 = Free)
* State - State location in Malaysia (Refer to StateLabels dictionary)
* RescuerID - Unique hash ID of rescuer
* VideoAmt - Total uploaded videos for this pet
* PhotoAmt - Total uploaded photos for this pet
* Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.


In [ ]:
import pandas as pd

In [5]:
# download_dir = './tutorial'
# zip_file = 'https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip'
# !wget 'https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip'
# !unzip ./petfinder_for_tutorial.zip

dataset_path = './petfinder_for_tutorial'
train_data = pd.read_csv(f'{dataset_path}/train.csv', index_col=0)
test_data = pd.read_csv(f'{dataset_path}/test.csv', index_col=0)
label_col = 'AdoptionSpeed'
print(len(train_data),len(test_data))
train_data.head()

600 100


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,Images
0,2,Yumi Hamasaki,4,292,265,2,1,5,7,2,...,1,0,41326,bcc4e1b9557a8b3aaf545ea8e6e86991,0,I rescued Yumi Hamasaki at a food stall far aw...,7d7a39d71,3.0,0,images/7d7a39d71-1.jpg
1,2,Nene/ Kimie,12,285,0,2,5,6,7,2,...,1,0,41326,f0450bf0efe0fa3ff9321d0b827b1237,0,Has adopted by a friend with new pet name Kimie,0e107c82f,3.0,0,images/0e107c82f-1.jpg
2,2,Mattie,12,266,0,2,1,7,0,2,...,1,0,41401,9b52af6d48a4521fd01d4028eb5879a3,0,I rescued Mattie with a broken leg. After surg...,1a8fd6707,5.0,0,images/1a8fd6707-1.jpg
3,1,NaN,1,189,307,2,1,2,0,2,...,1,0,41401,88da1210e021a5cf43480b074778f3bc,0,She born on 30 September . I really hope the a...,bca8b44ae,3.0,0,images/bca8b44ae-1.jpg
4,2,Coco,6,276,285,2,2,4,7,2,...,1,100,41326,227d7b1bcfaffb5f9882bf57b5ee8fab,0,Calico Tame and easy going Diet RC Kitten Supp...,2def67952,1.0,0,images/2def67952-1.jpg


In [32]:
#remove not help columns: RescuerID, Images, PetID
print(train_data.columns)
rm=['Name','RescuerID','PetID','Images','Description']
tra=train_data[[i for i in train_data.columns if not i in rm]]
test=test_data[[i for i in test_data.columns if not i in rm]]
print(tra.columns)

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
       'Images'],
      dtype='object')
Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')


In [59]:
# tra['Age']
scale

['Age', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt', 'AdoptionSpeed']

In [63]:
# data = [
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ]
# df = spark.createDataFrame(data, ["feature1", "vector", "category"])
# category_indexer = StringIndexer(inputCol="category", outputCol="category_index")


tra_df=spark.createDataFrame(list(tra.itertuples(index=False, name=None)), tra.columns.tolist())
test_df=spark.createDataFrame(list(test.itertuples(index=False, name=None)), tra.columns.tolist())

cate=['Type','Breed1','Breed2','Gender','Color1','Color2','Color3','MaturitySize','FurLength','Vaccinated','Dewormed','Sterilized','Health']
category_indexer=StringIndexer(inputCols=cate, outputCols=[i+'_' for i in cate])

scale=[i for i in tra.columns.tolist() if i not in cate]
scaler =[]
for i in scale:
    scaler.append(StandardScaler(inputCol=i, outputCol=i+'_', withStd=True, withMean=False))
# scaler =StandardScaler(inputCols=scale, outputCols=scale, withStd=True, withMean=False)

In [61]:
tra_df.columns

['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 'AdoptionSpeed']

In [66]:
assembler = VectorAssembler(inputCols=tra_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(tra_df)

predictions = model.transform(tra_df)
predictions.show()

+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|Type|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|VideoAmt|PhotoAmt|AdoptionSpeed|Type_|Breed1_|Breed2_|Gender_|Color1_|Color2_|Color3_|MaturitySize_|FurLength_|Vaccinated_|Dewormed_|Sterilized_|Health_|            features|prediction|
+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|   2|  4|   292|   265|     2|     1|     5| 

to predict test dataset

In [68]:
# training dataset performance
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7532297435812325


0.7532297435812325

In [69]:
assembler = VectorAssembler(inputCols=test_df.columns,outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[ category_indexer, assembler, kmeans])
model = pipeline.fit(test_df)

predictions = model.transform(test_df)
predictions.show()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|Type|Age|Breed1|Breed2|Gender|Color1|Color2|Color3|MaturitySize|FurLength|Vaccinated|Dewormed|Sterilized|Health|Quantity|Fee|State|VideoAmt|PhotoAmt|AdoptionSpeed|Type_|Breed1_|Breed2_|Gender_|Color1_|Color2_|Color3_|MaturitySize_|FurLength_|Vaccinated_|Dewormed_|Sterilized_|Health_|            features|prediction|
+----+---+------+------+------+------+------+------+------------+---------+----------+--------+----------+------+--------+---+-----+--------+--------+-------------+-----+-------+-------+-------+-------+-------+-------+-------------+----------+-----------+---------+-----------+-------+--------------------+----------+
|   2| 24|   265|   288|     1|     1|     5| 